In [1]:
import os
import itk
import sys
from ipywidgets import interact, fixed
from ipywidgets import interactive
import ipywidgets as widgets
from itkwidgets import view
import numpy as np
from matplotlib import pyplot as plt

In [2]:
FileName='CU102973_TLC.img.gz'
Image=itk.imread(FileName,itk.F)

In [3]:
PixelType=itk.F
ImageType=itk.Image[PixelType,3]

In [4]:
def flip_image(image):
    PixelType=itk.F
    ImageType=itk.Image[PixelType,3]
    flipAxes=(False,False,True)
        
    flipfilter=itk.FlipImageFilter[ImageType].New()
    flipfilter.SetInput(image)
    flipfilter.SetFlipAxes(flipAxes)
    flipfilter.Update()
    return flipfilter.GetOutput()

In [5]:
flipped=flip_image(Image)

In [6]:
view(flipped,cmap='Grayscale')

Viewer(cmap='Grayscale', gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Obje…

In [7]:
def flatten_image(input_image):
    """Flattens image into a 1D array.
    
    Args:
      input_image (itk.Image): Input image
    Returns:
      np_image_as_array (np.array): 1D array representing a image
      
    """
    feat_img = itk.GetArrayFromImage(input_image)
    np_image_as_array = feat_img.flatten()  
   
    return np_image_as_array

In [8]:
confidence_connected = itk.ConfidenceConnectedImageFilter.New(flipped)

In [9]:
confidence_connected.SetMultiplier(2.3)
confidence_connected.SetNumberOfIterations(5)
confidence_connected.SetInitialNeighborhoodRadius(3)
confidence_connected.SetReplaceValue(2000)

In [10]:
flipped.TransformPhysicalPointToIndex((152,119,-1.4))

itkIndex3 ([251, 197, 584])

In [11]:
confidence_connected.SetSeed([251,197,584])

In [12]:
confidence_connected.Update()
# 

In [13]:
view(confidence_connected.GetOutput())

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageSS3; proxy of <Swig Object of type 'itkIm…

In [15]:
imag_cc=confidence_connected.GetOutput()

In [16]:
def rescale_cc(image):
    InputPixelType=itk.SS
    OutputPixelType=itk.US
    InputImageType=itk.Image[InputPixelType,3]
    OutputImageType=itk.Image[OutputPixelType,3]
    rescaler=itk.RescaleIntensityImageFilter[InputImageType,OutputImageType].New()
    rescaler.SetInput(image)
    rescaler.Update()
    rescaler.SetOutputMinimum(0)
    rescaler.SetOutputMaximum(2000)
    return rescaler.GetOutput()

In [17]:
def rescale_to_float(image):
    arr_cc=itk.GetArrayFromImage(imag_cc)
    image_cc_ff = itk.GetImageFromArray(arr_cc.astype(np.float32),is_vector=False)
    return image_cc_ff

In [18]:
imag_ff=rescale_to_float(imag_cc)

In [30]:
def draw_edge(image):
    edgefilter=itk.SobelEdgeDetectionImageFilter[ImageType,ImageType].New(image)
    edgefilter.Update()
    rescaler=itk.RescaleIntensityImageFilter[ImageType,ImageType].New()
    rescaler.SetInput(edgefilter.GetOutput())
    
    rescaler.SetOutputMinimum(0)
    rescaler.SetOutputMaximum(2000)
    rescaler.Update()
    return rescaler.GetOutput()

In [31]:
image_edge=draw_edge(imag_ff)

In [32]:
view(image_edge)

Viewer(gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Object of type 'itkIma…